# Part 6: 고급 - 실제 AgentReview 사용 (선택)

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. 실제 AgentReview 프레임워크 사용
2. ICLR 논문으로 5단계 Peer Review 시뮬레이션
3. 다양한 리뷰어 페르소나 비교 (malicious, benign 등)

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치 + AgentReview 클론
if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install google-generativeai python-dotenv openai -q
    !git clone https://github.com/ahren09/agentreview.git 2>/dev/null || echo "  ✓ AgentReview already cloned"
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")
    
    # 로컬에서 AgentReview 클론
    import subprocess
    if not os.path.exists("agentreview"):
        print("   📥 AgentReview 클론 중...")
        subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git"], capture_output=True)
    else:
        print("   ✓ AgentReview already exists")

In [ ]:
# Cell 3: API Key 로딩
import os

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')  # AgentReview는 OpenAI 필요
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        GEMINI_API_KEY = None
        OPENAI_API_KEY = None
        print("⚠️ Colab Secrets에 API KEY를 추가하세요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        if GEMINI_API_KEY or OPENAI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 API KEY를 추가하세요")
    except ImportError:
        GEMINI_API_KEY = None
        OPENAI_API_KEY = None

os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY or ''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''

print(f"\n🔑 Gemini: {'설정됨' if GEMINI_API_KEY else '없음'}")
print(f"🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음 (AgentReview 제한)'}")
print("\n⚠️ 참고: 실제 AgentReview는 OpenAI API를 사용합니다.")
print("   Gemini만으로 실행하려면 notebook 4를 사용하세요.")

---
## 1. AgentReview 5단계 파이프라인

### EMNLP 2024 AgentReview

실제 ICLR peer review 프로세스를 LLM으로 시뮬레이션:

**Phase I: Reviewer Assessment**
- 3명의 리뷰어가 독립적으로 논문 평가
- 각자 점수 + 코멘트 작성

**Phase II: Author-Reviewer Discussion**
- 저자가 리뷰어 코멘트에 rebuttal 제출
- 리뷰어 우려사항 대응

**Phase III: Reviewer-AC Discussion**
- Area Chair(AC)가 리뷰어 간 토론 중재
- 리뷰어들이 초기 평가 업데이트

**Phase IV: Meta-Review**
- AC가 모든 토론 종합
- 최종 메타리뷰 작성

**Phase V: Paper Decision**
- AC가 Accept/Reject 결정
- ICLR 실제 acceptance rate (32%) 적용

---
## 2. 샘플 논문 준비

### Option A: 본인 논문으로 테스트 (간단)

ICLR 데이터 없이도 Phase I (리뷰어 평가)만 실행 가능

In [ ]:
# Cell 6: 샘플 논문 텍스트
sample_paper_text = """
TITLE: Do Large Language Models Exhibit Human-like Cognitive Biases?

ABSTRACT:
Large language models (LLMs) have shown remarkable capabilities in various 
cognitive tasks. This paper investigates whether LLMs exhibit human-like 
cognitive biases, specifically focusing on anchoring effects and confirmation 
bias. We design a series of experiments adapted from classical psychology 
studies and evaluate GPT-4 and Claude on these tasks.

METHODS:
We adapted 5 classical cognitive bias experiments from the psychology literature:
1. Anchoring effect (Tversky & Kahneman, 1974)
2. Confirmation bias (Wason, 1960)
3. Framing effect (Kahneman & Tversky, 1981)
4. Availability heuristic (Tversky & Kahneman, 1973)
5. Hindsight bias (Fischhoff, 1975)

Each experiment was administered to GPT-4 and Claude-3 with 100 variations.
We measured bias magnitude using effect size (Cohen's d) and compared to
human baseline data from original studies.

RESULTS:
- Anchoring effect: d=0.8 (GPT-4), d=0.6 (Claude) vs d=1.2 (human)
- Confirmation bias: Present in both models, magnitude similar to humans
- Framing effect: Weaker than humans (d=0.3 vs d=0.9)

DISCUSSION:
Our findings suggest LLMs exhibit cognitive biases but with different magnitudes.
This has implications for using LLMs as cognitive models and for AI safety.
Limitations include prompt sensitivity and lack of mechanistic explanation.
"""

print("✅ 샘플 논문 준비 완료")
print(f"   길이: {len(sample_paper_text)} 문자")

### Option B: ICLR 실제 논문 사용 (고급)

**필요사항:**
1. [AgentReview_Paper_Data.zip](https://www.dropbox.com/scl/fi/l17brtbzsy3xwflqd58ja/AgentReview_Paper_Data.zip?rlkey=vldiexmgzi7zycmz7pumgbooc&st=b6g3nkry&dl=0) 다운로드
2. `data/` 폴더에 압축 해제

```bash
# 로컬 환경에서:
unzip AgentReview_Paper_Data.zip -d data/
```

포함된 논문: ICLR 2020-2023 논문 PDF + 실제 리뷰 데이터

In [ ]:
# Cell 7: 데이터 존재 여부 확인
import os

data_dir = "data"
has_data = os.path.exists(data_dir) and len(os.listdir(data_dir)) > 0

if has_data:
    print("✅ ICLR 데이터 발견!")
    print(f"   폴더: {data_dir}")
    print(f"   파일 수: {len(os.listdir(data_dir))}")
    USE_REAL_DATA = True
else:
    print("⚠️ ICLR 데이터 없음")
    print("   Option A (샘플 논문 텍스트)로 진행합니다.")
    print("")
    print("   실제 데이터 사용하려면:")
    print("   1. Dropbox에서 AgentReview_Paper_Data.zip 다운로드")
    print("   2. data/ 폴더에 압축 해제")
    USE_REAL_DATA = False

---
## 3. AgentReview 실행

### 리뷰어 페르소나 설정

다양한 리뷰어 조합 실험 가능

In [ ]:
# Cell 9: 리뷰어 설정 정의

# 기본 설정: 모든 리뷰어가 일반적
baseline_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 설정 1: 악의적 리뷰어 1명
malicious_setting = {
    "reviewer": ["malicious", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 설정 2: 호의적 리뷰어 1명
benign_setting = {
    "reviewer": ["benign", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("✅ 리뷰어 설정 정의 완료")
print("\n사용 가능한 페르소나:")
print("  - BASELINE: 일반 리뷰어")
print("  - malicious: 악의적 리뷰어 (낮은 점수 선호)")
print("  - benign: 호의적 리뷰어 (높은 점수 선호)")

### Option A: 간단 버전 (Gemini 사용)

ICLR 데이터 없이 Phase I만 실행

In [ ]:
# Cell 10: Gemini로 간단 리뷰 (notebook 4와 동일)
import google.generativeai as genai
from dataclasses import dataclass
from typing import List, Dict

@dataclass
class ReviewerPersona:
    name: str
    expertise: str
    style: str
    focus: List[str]

class SimpleReviewer:
    def __init__(self, persona: ReviewerPersona):
        self.persona = persona
        if GEMINI_API_KEY:
            genai.configure(api_key=GEMINI_API_KEY)
            self.model = genai.GenerativeModel('gemini-1.5-flash')
        else:
            self.model = None
    
    def review(self, paper_text: str) -> Dict:
        if not self.model:
            return {"error": "API key 필요"}
        
        prompt = f"""You are {self.persona.name}, an expert in {self.persona.expertise}.
Style: {self.persona.style}
Focus: {', '.join(self.persona.focus)}

Review this paper:
1. Score (1-10)
2. Summary
3. Strengths
4. Weaknesses
5. Recommendation

PAPER:
{paper_text[:8000]}
"""
        
        response = self.model.generate_content(prompt)
        return {
            "reviewer": self.persona.name,
            "review": response.text
        }

if GEMINI_API_KEY:
    # 3명의 리뷰어 생성
    personas = [
        ReviewerPersona("Dr. ML Expert", "Machine Learning", "rigorous", ["methodology"]),
        ReviewerPersona("Dr. Psychologist", "Psychology", "constructive", ["theory"]),
        ReviewerPersona("Dr. AI Safety", "AI Safety", "critical", ["implications"])
    ]
    
    reviewers = [SimpleReviewer(p) for p in personas]
    
    print("📝 리뷰 시작...\n")
    for reviewer in reviewers:
        review = reviewer.review(sample_paper_text)
        print(f"\n{'='*60}")
        print(f"👤 {review['reviewer']}")
        print(f"{'='*60}")
        print(review['review'][:1000])
        print("...\n")
else:
    print("⚠️ Gemini API key 필요")

### Option B: 실제 AgentReview 사용 (OpenAI + 데이터 필요)

5단계 전체 파이프라인 실행

In [ ]:
# Cell 11: AgentReview 모듈 import
if USE_REAL_DATA and OPENAI_API_KEY:
    # AgentReview 경로 추가
    import sys
    sys.path.insert(0, os.path.abspath("agentreview"))
    
    try:
        from agentreview.environments import PaperReview
        from agentreview.paper_review_arena import PaperReviewArena
        from agentreview.paper_review_settings import get_experiment_settings
        from agentreview.utility.experiment_utils import initialize_players
        from agentreview.utility.utils import get_paper_decision_mapping
        
        print("✅ AgentReview 모듈 로딩 완료")
        AGENTREVIEW_AVAILABLE = True
    except Exception as e:
        print(f"⚠️ AgentReview 모듈 로딩 실패: {e}")
        print("   간단 버전(Option A)을 사용하세요.")
        AGENTREVIEW_AVAILABLE = False
else:
    print("⚠️ 실제 AgentReview 사용 불가")
    if not USE_REAL_DATA:
        print("   이유: ICLR 데이터 없음")
    if not OPENAI_API_KEY:
        print("   이유: OpenAI API key 없음")
    AGENTREVIEW_AVAILABLE = False

In [ ]:
# Cell 12: 실제 AgentReview 실행
if AGENTREVIEW_AVAILABLE:
    from argparse import Namespace
    
    # 설정
    args = Namespace(
        openai_client_type='openai',
        conference='ICLR2024',
        num_reviewers_per_paper=3,
        model_name='gpt-4o-mini',  # 비용 절감
        output_dir='outputs',
        data_dir='./data',
        acceptance_rate=0.32,
        skip_logging=False,
        task='paper_review'
    )
    
    # 논문 ID 매핑
    paper_id2decision, paper_decision2ids = get_paper_decision_mapping(
        args.data_dir, 
        args.conference
    )
    
    # 샘플 논문 ID (demo.ipynb에서 사용하는 것)
    sample_paper_ids = [39, 247]  # 2개만 테스트
    
    print(f"📝 {len(sample_paper_ids)}개 논문 리뷰 시작...\n")
    
    for paper_id in sample_paper_ids:
        paper_decision = paper_id2decision[paper_id]
        
        # 설정 선택 (malicious vs baseline)
        experiment_setting = get_experiment_settings(
            paper_id=paper_id,
            paper_decision=paper_decision,
            setting=malicious_setting  # <- 여기서 설정 변경 가능
        )
        
        print(f"\n{'='*60}")
        print(f"Paper ID: {paper_id}")
        print(f"실제 결과: {paper_decision}")
        print(f"{'='*60}\n")
        
        # 플레이어 초기화
        players = initialize_players(
            experiment_setting=experiment_setting, 
            args=args
        )
        
        # 환경 생성
        env = PaperReview(
            player_names=[p.name for p in players],
            paper_decision=paper_decision,
            paper_id=paper_id,
            args=args,
            experiment_setting=experiment_setting
        )
        
        # Arena 실행
        arena = PaperReviewArena(
            players=players, 
            environment=env, 
            args=args
        )
        
        # 비대화형 모드로 실행
        arena.launch_cli(interactive=False)
        
        print("\n✅ 리뷰 완료\n")
else:
    print("⚠️ 실제 AgentReview를 사용할 수 없습니다.")
    print("   Option A (간단 버전)을 사용하세요.")

---
## 4. 결과 분석

In [ ]:
# Cell 13: 결과 확인
if AGENTREVIEW_AVAILABLE:
    print("📊 생성된 리뷰 확인:")
    print(f"   출력 폴더: {args.output_dir}")
    print("")
    print("다음 파일들이 생성되었습니다:")
    print("  - Phase I: 리뷰어 평가")
    print("  - Phase II: Author rebuttal")
    print("  - Phase III: 리뷰어-AC 토론")
    print("  - Phase IV: Meta-review")
    print("  - Phase V: 최종 결정")
    
    # 출력 파일 목록
    import os
    if os.path.exists(args.output_dir):
        files = os.listdir(args.output_dir)
        print(f"\n생성된 파일 수: {len(files)}")
        for f in files[:5]:
            print(f"  - {f}")
else:
    print("결과 없음")

---
## 5. 실험: 다양한 설정 비교

In [ ]:
# DIY Cell 14: 설정 비교 실험

# TODO: 다양한 설정으로 실험해보세요

# 실험 아이디어:
# 1. baseline vs malicious: 악의적 리뷰어의 영향?
# 2. baseline vs benign: 호의적 리뷰어의 영향?
# 3. 3명 모두 malicious: 극단적인 케이스

settings_to_compare = {
    "baseline": baseline_setting,
    "malicious_1": malicious_setting,
    "benign_1": benign_setting
}

# 각 설정으로 동일 논문 리뷰
# 결과 비교 분석

---
## 6. 토론

### 토론 질문

1. **리뷰어 페르소나의 영향**
   - malicious 리뷰어가 있을 때 최종 결정이 바뀌나요?
   - benign 리뷰어의 효과는?

2. **5단계 파이프라인의 가치**
   - Rebuttal이 리뷰 점수를 바꾸나요?
   - AC 토론이 편향을 줄이나요?

3. **실제 연구 적용**
   - Peer review 시스템 개선에 어떻게 활용?
   - 논문 투고 전 사전 테스트?
   - 리뷰어 배정 최적화?

### 참고 자료

- 📄 [AgentReview 논문](https://arxiv.org/abs/2406.12708)
- 🌐 [AgentReview 웹사이트](https://agentreview.github.io/)
- 💻 [GitHub 저장소](https://github.com/Ahren09/AgentReview)
- 💡 [HuggingFace 데모](https://huggingface.co/spaces/Ahren09/AgentReview)